<a href="https://colab.research.google.com/github/drfperez/openair/blob/main/XEMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import time
import urllib.parse
from google.colab import files
from datetime import datetime

def descarregar_meteocat(any_seleccionat=None, codi_estacio=None, codi_variable=None):
    """
    Descarrega dades de la XEMA (Meteocat).
    Endpoint: https://analisi.transparenciacatalunya.cat/resource/nzvn-apee.json
    """
    base_url = "https://analisi.transparenciacatalunya.cat/resource/nzvn-apee.json"
    limit = 50000
    offset = 0
    tots_els_dades = []

    # Construir condicions SoQL
    condicions = []

    # Filtre per Estació (Ex: 'X4' és Barcelona - Raval, 'D5' és Barcelona - Zoo)
    if codi_estacio:
        condicions.append(f"codi_estacio='{codi_estacio}'")

    # Filtre per Variable (Ex: 32=Temp, 33=Humitat, 35=Precipitació)
    if codi_variable:
        condicions.append(f"codi_variable='{codi_variable}'")

    # Filtre per Data (Any)
    if any_seleccionat:
        # A Meteocat la columna sol ser 'data_lectura'
        data_inici = f"{any_seleccionat}-01-01T00:00:00"
        data_fi = f"{any_seleccionat+1}-01-01T00:00:00"
        condicions.append(f"data_lectura >= '{data_inici}' AND data_lectura < '{data_fi}'")

    where_clause = " AND ".join(condicions) if condicions else "1=1"

    print("🚀 Iniciant descàrrega de dades METEOCAT (XEMA)...")
    print(f"   Filtres -> Any: {any_seleccionat}, Estació: {codi_estacio}, Variable: {codi_variable}")

    while True:
        # Ordenem per data_lectura
        query = f"?$where={where_clause}&$limit={limit}&$offset={offset}&$order=data_lectura ASC"
        url_neta = base_url + urllib.parse.quote(query, safe='?=&-:T')

        try:
            df_bloc = pd.read_json(url_neta)

            if df_bloc.empty:
                break

            tots_els_dades.append(df_bloc)
            offset += limit

            # Mostrem progrés
            if 'data_lectura' in df_bloc.columns:
                ultima_data = df_bloc['data_lectura'].iloc[-1]
                print(f"📥 {offset} registres processats... Data actual: {str(ultima_data)[:10]}")
            else:
                print(f"📥 {offset} registres processats...")

            time.sleep(0.3)

        except Exception as e:
            print(f"⚠️ Error durant la descàrrega: {e}")
            break

    if tots_els_dades:
        df_final = pd.concat(tots_els_dades, ignore_index=True)

        # Convertim columnes de data
        if 'data_lectura' in df_final.columns:
            df_final['data_lectura'] = pd.to_datetime(df_final['data_lectura'])

        print(f"\n✅ ÈXIT: {len(df_final)} registres descarregats.")
        if not df_final.empty and 'data_lectura' in df_final.columns:
            print(f"📅 Període: {df_final['data_lectura'].min().date()} fins a {df_final['data_lectura'].max().date()}")

        # Nom de fitxer dinàmic
        nom_fitxer = f"Meteocat_XEMA"
        if codi_estacio: nom_fitxer += f"_{codi_estacio}"
        if codi_variable: nom_fitxer += f"_Var{codi_variable}"
        if any_seleccionat: nom_fitxer += f"_{any_seleccionat}"
        nom_fitxer += ".csv"

        df_final.to_csv(nom_fitxer, index=False)
        files.download(nom_fitxer)

        return df_final
    else:
        print("❌ No s'han trobat dades amb aquests filtres.")
        return None

# --- INTERACCIÓ AMB L'USUARI ---

print("--- DESCARREGADOR DADES METEOCAT (XEMA) ---")
print("ℹ️  Guia ràpida de codis:")
print("   VARIABLES COMUNES: 32 (Temp), 33 (Humitat), 35 (Pluja), 30 (Vent Vel.), 31 (Vent Dir.)")
print("   EXEMPLES ESTACIONS: X4 (Bcn-Raval), D5 (Bcn-Zoo), WU (Badalona), Y9 (Girona)")
print("----------------------------------------------------")

any_usuari = input("Introdueix l'any (deixa buit per tots): ").strip()
estacio_usuari = input("Introdueix Codi Estació (ex: X4, D5...): ").strip().upper()
variable_usuari = input("Introdueix Codi Variable (ex: 32, 35...): ").strip()

# Processar inputs
any_usuari = int(any_usuari) if any_usuari.isdigit() else None
estacio_usuari = estacio_usuari if estacio_usuari else None
variable_usuari = variable_usuari if variable_usuari else None

# Executar funció
df_resultat = descarregar_meteocat(any_usuari, estacio_usuari, variable_usuari)

--- DESCARREGADOR DADES METEOCAT (XEMA) ---
ℹ️  Guia ràpida de codis:
   VARIABLES COMUNES: 32 (Temp), 33 (Humitat), 35 (Pluja), 30 (Vent Vel.), 31 (Vent Dir.)
   EXEMPLES ESTACIONS: X4 (Bcn-Raval), D5 (Bcn-Zoo), WU (Badalona), Y9 (Girona)
----------------------------------------------------
🚀 Iniciant descàrrega de dades METEOCAT (XEMA)...
   Filtres -> Any: 2025, Estació: X4, Variable: None
📥 50000 registres processats... Data actual: 2025-03-07
📥 100000 registres processats... Data actual: 2025-05-11
📥 150000 registres processats... Data actual: 2025-07-15
📥 200000 registres processats... Data actual: 2025-09-18
📥 250000 registres processats... Data actual: 2025-11-22
📥 300000 registres processats... Data actual: 2025-12-31

✅ ÈXIT: 280012 registres descarregats.
📅 Període: 2025-01-01 fins a 2025-12-31


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>